In [1]:
# For downloading data
import sxs

# For interacting with the data
import h5py
import numpy as np
from matplotlib import pyplot as plt
import json
import pandas as pd

#Other libraries
from lmfit import Model, minimize, Parameters
import qnm

In [2]:
#Initializing arrays that will be used for the fitting function
waveforms = []
names = []
spins = []
masses = []

#Loop through the entire SXS BBH catalog
for i in range(2300):
    #Catch exception block is here because I didnt want to check how many BBH simulations there are, I know its over 2265, so I put 2300.
    try:
        #Change the format of i from say, 1 to 0001 as the SXS BBH catalog has its files names in that format
        i = '{0:04}'.format(i)

        #Load the metadata to get the spin and mass information, spin is most important right now
        mtdt = sxs.load(f"SXS:BBH:{i}/Lev/metadata.json", download = False)
        spin = mtdt.remnant_dimensionless_spin[2]
        mass = mtdt.remnant_mass
        
        #Most of the BBHs with random spin orientations will converge to Chi = 0.69, which is what we're looking for to compare the overtone numbers.
        if 0.65 <= spin <= 0.75:
            #If the spin matches, then load the large waveform rhOVerM file, it will download if not already stored locally and each is about 80-100 mb
            #Store all the information of masses, spins, etc to the arrays
            names.append(i)
            spins.append(spin)
            masses.append(mass)
            w = sxs.load(f"SXS:BBH:{i}/Lev/rhOverM", extrapolation_order=2, download = False)
            waveforms.append([i,w])
    except:
        pass

Found the following files to load from the SXS catalog:
    SXS:BBH:0001v6/Lev5/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0001v6/Lev5/rhOverM_Asymptotic_GeometricUnits_CoM.h5
Found the following files to load from the SXS catalog:
    SXS:BBH:0002v7/Lev6/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0002v7/Lev6/rhOverM_Asymptotic_GeometricUnits_CoM.h5
Found the following files to load from the SXS catalog:
    SXS:BBH:0003v5/Lev5/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0003v5/Lev5/rhOverM_Asymptotic_GeometricUnits_CoM.h5
Found the following files to load from the SXS catalog:
    SXS:BBH:0004v5/Lev6/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0005v5/Lev5/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0006v5/Lev5/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0007v

In [3]:
times_after = np.linspace(0,25,6)
times_after

array([ 0.,  5., 10., 15., 20., 25.])

In [4]:
import json

timenameCof = []

for i in range(len(times_after)):
    with open('Unorganized_files/All_Coefficients_%iM.txt'%(times_after[i]), 'r') as f:
        coef = json.load(f)
    timenameCof.append(coef)

In [27]:
oldNames = [timenameCof[0][i][0] for i in range(len(timenameCof[0]))]

In [32]:
len(timenameCof[0])

589

In [35]:
nameindex = list(set(names) ^ set(oldNames))
indexes = [names.index(x) for x in nameindex]

In [34]:
# assuming list1 is the list with 589 lists and list2 is the list with 595 lists
indexes2 = []
for i, item in enumerate(names):
    if item not in oldNames:
        indexes.append(i)

In [43]:
oldNames.index('0309')

101

In [37]:
nameindex

['0447', '0923', '0917', '0312', '0310', '0632']

In [36]:
print(indexes)
print(indexes2)

[151, 345, 344, 103, 102, 206]
[]


In [44]:
for i in indexes:
    print(spins[i])
    print(masses[i])
    print()

0.654722609665
0.954024086784

0.688028319113
0.950857134671

0.680197902659
0.958467587056

0.697209331299
0.95146782214

0.680979004173
0.952532925197

0.658332049514
0.953995670402



In [45]:
timenameCof[0]

[['0001',
  [0.9507659091822734,
   -0.6690622375816772,
   -3.9228109447655495,
   -1.72120372037849,
   -11.387638679110603,
   0.9466352794638069,
   26.874528890793865,
   0.895424759729753,
   -45.01705744519322,
   1.065717858977951,
   -45.26917443609964,
   -1.855112213468027,
   -24.094928532850002,
   1.4828232179176455,
   -5.2951103617624815,
   4.73955974352272,
   0.0007970050248157826]],
 ['0002',
  [-0.9834868613349507,
   -0.5339653667044688,
   2.2351569457094307,
   -1.6152137310014973,
   12.783467187973653,
   0.010305429472512334,
   -74.00142844822192,
   0.04373883436138682,
   204.36979979367862,
   0.28259625950493467,
   277.4676483844329,
   -2.641837089270255,
   182.12463147559964,
   0.6657027415012502,
   46.73411021607431,
   3.8970274311048856,
   -0.007070033873061958]],
 ['0003',
  [-0.8244123068668665,
   -0.7475558052907568,
   2.999894976040874,
   -1.7599968576474614,
   7.048650284111096,
   0.9955839321286201,
   -14.441364347817439,
   1.09097

In [49]:
old_spins = [spin for i, spin in enumerate(spins) if i not in indexes]
old_masses = [mass for i, mass in enumerate(masses) if i not in indexes]
old_names = oldNames

In [53]:
timenameCof[0]

[['0001',
  [0.9507659091822734,
   -0.6690622375816772,
   -3.9228109447655495,
   -1.72120372037849,
   -11.387638679110603,
   0.9466352794638069,
   26.874528890793865,
   0.895424759729753,
   -45.01705744519322,
   1.065717858977951,
   -45.26917443609964,
   -1.855112213468027,
   -24.094928532850002,
   1.4828232179176455,
   -5.2951103617624815,
   4.73955974352272,
   0.0007970050248157826]],
 ['0002',
  [-0.9834868613349507,
   -0.5339653667044688,
   2.2351569457094307,
   -1.6152137310014973,
   12.783467187973653,
   0.010305429472512334,
   -74.00142844822192,
   0.04373883436138682,
   204.36979979367862,
   0.28259625950493467,
   277.4676483844329,
   -2.641837089270255,
   182.12463147559964,
   0.6657027415012502,
   46.73411021607431,
   3.8970274311048856,
   -0.007070033873061958]],
 ['0003',
  [-0.8244123068668665,
   -0.7475558052907568,
   2.999894976040874,
   -1.7599968576474614,
   7.048650284111096,
   0.9955839321286201,
   -14.441364347817439,
   1.09097

In [61]:
len(timenameCof)

6

In [66]:
param_values_0M = [timenameCof[0][i][1] for i in range(len(timenameCof[0]))]
param_values_5M = [timenameCof[1][i][1] for i in range(len(timenameCof[1]))]
param_values_10M = [timenameCof[2][i][1] for i in range(len(timenameCof[2]))]
param_values_15M = [timenameCof[3][i][1] for i in range(len(timenameCof[3]))]
param_values_20M = [timenameCof[4][i][1] for i in range(len(timenameCof[4]))]
param_values_25M = [timenameCof[5][i][1] for i in range(len(timenameCof[5]))]

In [90]:
def get_amplitudes(param_values):
    pure_amplitudes = []

    for result_values in param_values:
        pure_amps = []
        result_values = np.abs(result_values)
        for i in range(8):
            pure_amp = abs(result_values[2 * i])
            pure_amps.append(pure_amp)
        pure_amplitudes.append(pure_amps)
    return pure_amplitudes

In [94]:
def get_phases(param_values):
    phases = []

    for result_values in param_values:
        phase_vals = [result_values[2 * i + 1] for i in range(8)]
        phases.append(phase_vals)
    return phases

In [102]:
data = {'Names': old_names, 'Spin': old_spins, 'Remnant_Mass': old_masses}
amplitudes = get_amplitudes(param_values_0M)
phases = get_phases(param_values_0M)
for i in range(8):
    data[f'C_{i}'] = [amp[i] for amp in amplitudes]
for i in range(8):
    data[f'phi_{i}'] = [phase[i] for phase in phases]
df = pd.DataFrame(data)

In [104]:
# param_values_list = [param_values_0M, param_values_5M, param_values_10M, param_values_15M, param_values_20M, param_values_25M]
# for i, param_values in enumerate(param_values_list):
#     data = {'Names': old_names, 'Spin': old_spins, 'Remnant_Mass': old_masses}
#     amplitudes = get_amplitudes(param_values)
#     phases = get_phases(param_values)
#     for j in range(8):
#         data[f'C_{j}'] = [amp[j] for amp in amplitudes]
#     for j in range(8):
#         data[f'phi_{j}'] = [phase[j] for phase in phases]
#     df = pd.DataFrame(data)
#     df.to_csv(f'Excitation_Coefficient_{i*5}M.csv', index=False)

In [105]:
df_list = []

for i in range(6):
    df = pd.read_csv(f'Excitation_Coefficient_{i*5}M.csv')
    df_list.append(df)

In [109]:
df_list[4]

,Names,Spin,Remnant_Mass,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,phi_0,phi_1,phi_2,phi_3,phi_4,phi_5,phi_6,phi_7
0,1,0.686462,0.951609,0.173527,0.178767,1.924066,9.895745,25.154711,31.771495,19.556671,4.774652,0.960949,-3.574421,-3.665957,-3.667426,-0.434161,-0.320438,-0.229183,2.942831
1,2,0.686448,0.951610,0.158689,0.773545,7.706844,38.753395,99.166961,126.270050,78.236441,19.198112,0.936933,-7.098410,-4.098509,-1.015951,2.191666,-0.849466,2.374325,5.540265
2,3,0.691447,0.951083,0.151393,0.097280,1.154479,6.091931,16.208192,21.184726,13.362908,3.318378,0.990473,-0.457071,-0.254243,-0.190754,-0.075706,0.045862,0.138645,0.168446
3,7,0.664091,0.955270,0.167010,0.061384,0.564550,3.034803,7.910222,10.264556,6.504516,1.625778,0.362473,2.584040,-3.457301,-3.338779,-0.037636,0.116991,0.239061,3.442856
4,8,0.664091,0.955267,0.166000,0.056861,0.487728,2.317005,5.651649,7.041232,4.327412,1.059999,-0.666634,-2.728721,-3.125930,-3.140018,0.055302,0.128015,0.179959,3.321494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,2153,0.661903,0.966646,0.135863,0.013365,0.075412,0.352173,0.839735,1.029289,0.625416,0.150920,-0.141580,36.046201,-0.221253,-0.024515,0.198941,0.401535,0.564401,0.660069
585,2154,0.670840,0.965718,0.138179,0.012922,0.082406,0.350130,0.776967,0.905595,0.530383,0.124352,0.641799,-0.913745,0.626228,0.912682,1.206576,4.611765,-1.452213,1.831530
586,2155,0.679580,0.964696,0.141337,0.009369,0.104917,0.510957,1.281840,1.644295,1.039614,0.259392,4.669186,-2.777183,0.975395,1.329054,1.649812,-1.230471,-1.030336,-0.913471
587,2156,0.695775,0.965339,0.139750,0.023107,0.066205,0.309402,0.756877,0.941495,0.577034,0.140194,0.409312,-1.012127,-1.412493,-1.064061,-0.757090,-0.499012,-0.296021,-0.180783
